In [24]:
import csv
import cv2
import matplotlib.pyplot as plt
from plotnine import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
%matplotlib inline
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [53]:
class MuraTrainingDataset(torch.utils.data.Dataset):
    
    def __init__(self,text_file,root_dir, transform = transforms.Compose([transforms.Resize((150,150)),
                                                                          transforms.ToTensor(),
                                                                          transforms.Normalize(
                                                                              mean = [0.24808845, 0.24808845, 0.24808845],
                                                                              std = [0.01409452, 0.01409452, 0.01409452])])):
        """
        Args:
            text_file(string): path to text file
            root_dir(string): directory with all train images
        """
        #File with the Path
        self.name_frame = pd.read_csv(text_file,sep=",",usecols=range(1),dtype = 'str',nrows = 10000)
        #File with labels
        self.label_frame = pd.read_csv(text_file,sep=",", usecols=[1], nrows=10000)
        self.root_dir = root_dir
        self.transform = transform
                                       
    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        #Pull image
        img_name = os.path.join(self.root_dir, self.name_frame.iloc[idx, 0])
        
        image = Image.open(img_name)
        image = image.convert('RGB')
        #Apply tensor transformations to images
        image = self.transform(image) 
        # I included labels in the train_image_path file in excel using a find("positive")
        labels = self.label_frame.iloc[idx,0] 
        #Formatting for labels
        labels = np.array(labels)
        labels = np.reshape(labels, (1,1))
        labels= torch.from_numpy(labels.astype('int'))

        sample = {'image': image, 'labels': labels}
        
        return sample

In [41]:
MuraTrainSet = MuraTrainingDataset(text_file ='/Users/JosephVele/MURA-v1.1/train_image_paths.csv',
                                   root_dir = '/Users/JosephVele')

MuraTrainLoader = torch.utils.data.DataLoader(MuraTrainSet,batch_size=100,shuffle=True, num_workers=0)

In [42]:
#Calculate mean &std
x=[]
for i_batch,sample_batched in enumerate(MuraTrainLoader,0):
    numpy_image = sample_batched['image'].numpy()
    x.append(np.mean(numpy_image, axis=(0,2,3)))

image_mean=np.mean(x, axis= (0))
image_std=np.std(x, axis=(0))

In [43]:
#Confirm images have been normalized appropriately with 0 mean and 1 std
print(image_mean)
print(image_std)

[-4.214048e-06 -4.214048e-06 -4.214048e-06]
[0.94273394 0.94273394 0.94273394]


In [46]:
# Define hyper-parameters
#******************************#
drp1 = .1 #Dropout Rate for Convolutional Layer
drp2 = .5 #Dropout Rate for Dense Layer
learning = .01 # Learning Rate 
moment =.9 #Momentum 

#******************************#

In [50]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, num_classes=2, drop1 = .1, drop2 = .5):
        super(ConvNet, self).__init__()
        # 3.1 Initialization
        #******************************#
        # 1 input image channel, 10 output channels, 1x1 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 10, kernel_size=1) #Output = 75 x 75 x 10 
        self.conv1_bn = nn.BatchNorm2d(10)
        
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=2) #Output =18x18x20
        self.conv2_bn = nn.BatchNorm2d(20)
        self.conv2_drop = nn.Dropout2d(p=drop1)
        
        self.conv3 = nn.Conv2d(20, 30, kernel_size=3)  #Output =8 x 8 x 30
        self.conv3_bn = nn.BatchNorm2d(30)
        self.conv3_drop = nn.Dropout2d(p=drop1)
        
        self.conv4 = nn.Conv2d(30, 40, kernel_size=3) #Output = 3 x 3 x 40 
        self.conv4_bn = nn.BatchNorm2d(40)
        self.conv4_drop = nn.Dropout2d(p=drop1)
        
        self.dense1 = nn.Linear(360, 50)
        self.dense1_drop = nn.Dropout2d(p=drop2) 
        self.dense2 = nn.Linear(50, 25)
        self.dense2_drop = nn.Dropout2d(p=drop2)
        self.dense3 = nn.Linear(25, 1)
                
        
        #******************************#
    def forward(self, x):
        # 3.2 Define Neural Network
        #******************************#
        
        #convolutional layers
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2_bn(self.conv2(x))), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3_bn(self.conv3(x))), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4_bn(self.conv4(x))), 2))
        
        #flatten
        x = x.view(-1, 360)
        
        #dense layers
        x = F.relu(self.dense1(x))
        x = self.dense1_drop(x)
        
        x = F.relu(self.dense2(x))
        x = self.dense2_drop(x)
        
        x = self.dense3(x)
        
        #******************************#
        return x
        
                       
model = ConvNet(num_classes=2, drop1 = .1, drop2 = .5).to(device)
# Note: what is the difference between 'same' and 'valid' padding? 
# Take a look at the outputs to understand the difference.

In [60]:
# 4.1 Define criterion and optimizer
#************************************#
import torch.optim as optim


criterion = nn.BCEWithLogitsLoss() #Binary Cross Entropy
optimizer = optim.SGD(model.parameters(), lr=learning, momentum=moment)

#************************************#

# 4.2 Train the model
# 4.3 Please store and print training and validation loss&accuracy after each epoch
#********************************************#

train_losses = []

test_losses = []

acc_data_train=[]
acc_data_test=[]

def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    for i, sample_batched in enumerate(MuraTrainLoader,1):
        inputs = sample_batched['image']
        labels = sample_batched['labels']
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        labels = labels.view(len(sample_batched['labels']),-1)
        loss = criterion(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.item()
        pred = (torch.sigmoid(outputs).data > 0.5).int()

        labels = labels.int()

        correct += (pred == labels).int().sum()


    
    #Print Results from first Epoch
    #Accuracy
    acc =  100.00 * float(correct) / float(len(MuraTrainLoader.dataset))
    acc_data_train.append(acc)
    #Average Loss
    train_loss=float(train_loss)/float(i)
    train_losses.append(train_loss)
    # print statistics        
    print('\nTrain Epoch:{} Accuracy: {:.4f} \t Average Loss: {:.4f}\n'.format(epoch,
            acc, train_loss))

In [ ]:
#Tuning Learning Rate
for i in [10,1,.1,.01]:
    for epoch in range(1,2):  # loop over the dataset multiple times
        model = ConvNet(num_classes=2, drop1 = .1, drop2 = .5).to(device)
        criterion = nn.BCEWithLogitsLoss() #Binary Cross Entropy
        optimizer = optim.SGD(model.parameters(), lr=i, momentum=moment)
        print('Learning Rate {}'.format(i) )
        train(epoch)

Learning Rate 10
